# Laboratorio 3. Clasificación multiclase.
Rúa Echalar Juan Manuel

In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [112]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Cargar el dataset
data = pd.read_csv('/content/drive/MyDrive/IA/SIS420/Clase_pasajero_test.csv')
# Eliminar la columna 'id' si no es relevante para el análisis
data.drop('#', axis=1, inplace=True)

# Codificar la columna 'Gender' usando LabelEncoder
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

# Codificar la columna 'Customer Type' usando LabelEncoder
data['Customer Type'] = label_encoder.fit_transform(data['Customer Type'])

# Codificar la columna 'Type of Travel' usando LabelEncoder
data['Type of Travel'] = label_encoder.fit_transform(data['Type of Travel'])

# Convertir 'satisfaction' a 1 para 'satisfied' y 0 para 'neutral or dissatisfaction'
data['satisfaction'] = (data['satisfaction'] == 'satisfied').astype(int)

# Reorganizar las columnas para poner 'Class' al final
class_col = data.pop('Class')
data['Class'] = class_col

# Guardar el dataset preprocesado en un nuevo archivo CSV
data.to_csv('dataset_preprocesado_numerico.csv', index=False)


In [113]:
# Cargar el dataset usando Pandas
data = pd.read_csv('dataset_preprocesado_numerico.csv')

# Eliminar la primera fila que contiene los encabezados de las columnas
data = data.iloc[1:]

# Convertir la columna 'Class' en valores numéricos
class_mapping = {'Business': 1, 'Eco': 2, 'Eco Plus': 3}
data['Class'] = data['Class'].map(class_mapping)

# Eliminar las filas que contienen valores faltantes
data.dropna(inplace=True)

# Convertir el DataFrame a una matriz NumPy si es necesario
data_np = data.to_numpy()

# Ahora puedes acceder a 'data_np' para obtener tus datos como una matriz NumPy


In [114]:
# La entrada es de 23 elemento contando con x0
input_layer_size  = 23

# 3 etiquetas (1, 2 o 3)
num_labels = 3

X = data_np[:20714, :-1]
y = data_np[:20714, -1]
X_test = data_np[20714:, :-1]
Y_test = data_np[20714:, -1]

m = y.size

In [115]:
print(X.shape)

(20714, 23)


In [116]:
print(X[0,:])
print(y)

[9.0035e+04 0.0000e+00 0.0000e+00 3.6000e+01 0.0000e+00 2.8630e+03
 1.0000e+00 1.0000e+00 3.0000e+00 1.0000e+00 5.0000e+00 4.0000e+00
 5.0000e+00 4.0000e+00 4.0000e+00 4.0000e+00 4.0000e+00 3.0000e+00
 4.0000e+00 5.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00]
[1. 2. 1. ... 1. 1. 2.]


In [117]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [118]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)
X_test_norm, mu_test,sigma_test = featureNormalize(X_test)

In [119]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
X = X_norm


In [120]:
print(X.shape)

(20714, 23)


In [121]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]


In [122]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [123]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """

    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [124]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [125]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(3, 24)


In [126]:
print(all_theta)

[[-1.24380805e+01 -1.10762221e-12  9.38562007e-14  2.31999303e-12
  -2.10965276e-12  4.31204928e-12 -4.29460399e-12 -6.00245078e-12
  -1.56788800e-12 -4.18145145e-12 -1.15608430e-12 -7.72923450e-12
  -8.38306519e-12 -9.24244050e-12 -1.11522693e-11 -7.36040980e-12
  -6.23807242e-12 -6.70685094e-12 -4.81152892e-12 -6.76108614e-12
  -9.18810818e-12  1.00662419e-12  1.12441198e-12 -9.49956618e-12]
 [-4.75404761e-02  2.15061407e-01  3.74346656e-02 -2.45180902e-01
   1.62071160e-01 -1.59152603e+00  1.08025391e+00 -8.28211341e-01
  -8.09138315e-03  2.66422725e-01  6.65390667e-02  7.08371446e-02
   5.83642908e-01  2.61043581e-01 -2.97751163e-01  2.86274672e-01
   1.30477036e-01  2.91524726e-01  1.93418852e-01  2.30371387e-01
  -5.27299334e-02 -5.27400931e-02  2.60484118e-02  3.78943276e-01]
 [-5.09480370e-01 -1.56865157e-01 -1.07624983e-03  4.11427996e-01
  -9.33802888e-02  1.14086415e+00 -7.69403643e-01  5.95753697e-01
   4.20298840e-02 -2.12645029e-01 -4.82298421e-02 -4.75805932e-02
  -3.783

In [127]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [128]:
print(X_norm)

[[ 0.66554022 -0.98248417 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]
 [-1.39339301  1.01782811  2.10230244 ... -0.38344123 -0.39313462
  -0.8821442 ]
 [ 0.34544138  1.01782811 -0.47566895 ... -0.38344123 -0.23317697
   1.13360152]
 ...
 [ 0.0501266  -0.98248417 -0.47566895 ... -0.27568503 -0.09987893
   1.13360152]
 [ 0.55280584 -0.98248417 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]
 [ 0.95022568  1.01782811 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]]


In [129]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print(X.shape)
print(X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

print(y[10:150])

(20714, 23)
(20714, 23)
[[ 0.66554022 -0.98248417 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]
 [-1.39339301  1.01782811  2.10230244 ... -0.38344123 -0.39313462
  -0.8821442 ]
 [ 0.34544138  1.01782811 -0.47566895 ... -0.38344123 -0.23317697
   1.13360152]
 ...
 [ 0.0501266  -0.98248417 -0.47566895 ... -0.27568503 -0.09987893
   1.13360152]
 [ 0.55280584 -0.98248417 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]
 [ 0.95022568  1.01782811 -0.47566895 ... -0.38344123 -0.39313462
   1.13360152]]
Precision del conjuto de entrenamiento: 77.82%
(140, 23)
(140, 24)
[1 1 1 1 2 1 2 2 1 1 2 2 1 2 2 1 2 1 2 2 1 2 2 2 2 1 1 1 2 2 2 2 1 2 2 1 2
 1 2 2 1 1 1 1 2 1 2 2 2 1 1 2 2 1 2 1 2 1 1 2 1 2 1 2 2 1 2 2 2 1 2 1 1 2
 2 2 1 2 1 1 2 1 1 1 2 2 1 2 1 2 2 2 2 1 1 1 2 2 1 1 2 1 2 2 2 2 1 2 1 1 1
 2 2 2 2 2 2 1 1 2 2 2 2 1 2 1 1 2 1 1 1 2 1 1 1 1 1 2 2 1]
[1. 1. 1. 1. 2. 2. 3. 2. 1. 1. 2. 2. 1. 2. 2. 1. 2. 1. 2. 1. 2. 2. 2. 2.
 2. 1. 1. 1. 2. 1. 2. 1. 1. 3. 2. 1. 2. 1. 2. 3. 1. 2. 2. 1

In [130]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X_test_norm[10:150, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(Y_test[10:150])

(20714, 23)
Precision del conjuto de entrenamiento: 77.82%
(140, 23)
(140, 24)
[1 2 2 1 1 2 2 1 2 2 1 1 1 2 2 2 1 1 1 1 2 1 1 2 2 1 1 1 2 2 1 2 1 1 1 1 1
 1 1 1 1 1 2 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 2 2 2 1 1 1 1 2 2 1 1 2 2 1 1 1
 1 1 1 1 2 1 1 2 2 2 2 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 1 2 1 2 2 2 2 1 2 2 2 2 2 1 1 2 1 2 1 2 2 1 2 1 1 2]
[1. 1. 2. 1. 1. 2. 2. 1. 2. 2. 1. 1. 1. 2. 2. 1. 1. 2. 1. 2. 2. 1. 2. 2.
 2. 1. 1. 1. 2. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 1. 1.
 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 3. 2. 1. 1. 2. 1. 2. 3. 1. 1. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 2. 2. 2. 1. 2. 1. 1. 1. 2. 1. 2. 1. 1. 2.
 1. 1. 3. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 3. 1. 1. 2. 1. 2. 2. 3.
 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 2. 2. 1. 2. 3. 1. 2.]


In [131]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X_test_norm[10:150, :].copy()
print(XPrueba.shape)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)

# Imprimir la clase predicha y la clase real
correctas = 0
total = len(Y_test[10:110])
for predicha, real in zip(p, y[10:110]):
    if predicha == real:
        correctas += 1
    print("Clase predicha para el pasajero: {}, Clase real: {}".format(predicha, real))

# Calcular y mostrar la precisión
precision = correctas / total
print("Correctas: {}/{}".format(correctas, total))


(20714, 23)
Precision del conjunto de entrenamiento: 77.82%
(140, 23)
Clase predicha para el pasajero: 1, Clase real: 1.0
Clase predicha para el pasajero: 2, Clase real: 1.0
Clase predicha para el pasajero: 2, Clase real: 1.0
Clase predicha para el pasajero: 1, Clase real: 1.0
Clase predicha para el pasajero: 1, Clase real: 2.0
Clase predicha para el pasajero: 2, Clase real: 2.0
Clase predicha para el pasajero: 2, Clase real: 3.0
Clase predicha para el pasajero: 1, Clase real: 2.0
Clase predicha para el pasajero: 2, Clase real: 1.0
Clase predicha para el pasajero: 2, Clase real: 1.0
Clase predicha para el pasajero: 1, Clase real: 2.0
Clase predicha para el pasajero: 1, Clase real: 2.0
Clase predicha para el pasajero: 1, Clase real: 1.0
Clase predicha para el pasajero: 2, Clase real: 2.0
Clase predicha para el pasajero: 2, Clase real: 2.0
Clase predicha para el pasajero: 2, Clase real: 1.0
Clase predicha para el pasajero: 1, Clase real: 2.0
Clase predicha para el pasajero: 1, Clase real